In [1]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = nfl.import_pbp_data(list(range(2018, 2024)), downcast=False, cache=False, alt_path=None)

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.


MemoryError: Unable to allocate 260. MiB for an array with shape (117, 291093) and data type float64

In [ ]:
#lastzn = nfl.import_pbp_data(list(range(2018, 2019)), downcast=False, cache=False, alt_path=None)

In [ ]:
#lastzn[(lastzn['game_id']=='2018_10_ARI_KC')&(lastzn['posteam']=='ARI')&(lastzn['pass']==1)][['desc','play_type']]

In [ ]:
df = df[df['season_type']=='REG']
df.loc[df['pass']==1, 'play_type'] = 'pass'
df.loc[df.rush==1, 'play_type'] = 'run'
df['total_plays'] = df['pass'] + df['rush']
df = df[(df['play_type']=='pass')|(df['play_type']=='run')]

In [ ]:
games = df.copy()

In [ ]:
# Group by 'game_id' and calculate both the sum and mean for 'pass', and any other aggregations as needed
games_grouped = games.groupby(['game_id','posteam']).agg(
    pass_total=('pass', 'sum'),
    pass_rate=('pass', 'mean'),
    pass_oe=('pass_oe', 'mean'),
    total_plays=('total_plays', 'sum'),
    total_line=('total_line', 'max'),
    spread_line=('spread_line', 'max'),
    week=('week', 'max'),
    season=('season', 'max'),

    #posteam=('posteam', 'max'),
    home_team=('home_team', 'max')).reset_index()  # reset_index() to turn the group labels back into columns if needed


In [ ]:
defense_grouped = games.groupby(['game_id','defteam']).agg(
    pass_total=('pass', 'sum'),
    pass_rate=('pass', 'mean'),
    pass_oe=('pass_oe', 'mean'),
    total_plays=('total_plays', 'sum'),
    total_line=('total_line', 'max'),
    spread_line=('spread_line', 'max'),
    week=('week', 'max'),
    season=('season', 'max'),
    posteam=('posteam', 'max'),
    home_team=('home_team', 'max')).reset_index()  # reset_index() to turn the group labels back into columns if needed

In [ ]:
defense_grouped

In [ ]:

def update_spread(games):
  # Create a new column 'pos_spread' to store the adjusted spread values
  games['pos_spread'] = np.where(games['posteam'] == games['home_team'], games['spread_line'], games['spread_line'] * -1)
  return games

games_grouped = update_spread(games_grouped.copy())
defenses_grouped = update_spread(defense_grouped.copy())

In [ ]:
games_grouped['pos_team_total'] = games_grouped['total_line']/2 + games_grouped['pos_spread']/2
defenses_grouped['pos_team_total'] = defenses_grouped['total_line']/2 + defenses_grouped['pos_spread']/2


In [ ]:
df = games_grouped.sort_values(['posteam','season','week'])

In [ ]:
defense_df = defenses_grouped.sort_values(['defteam','season','week'])

In [ ]:
def data_creator(df, trailing_weeks):
# Assuming df is your original dataframe with relevant columns and it's sorted by game date
# Calculate the trailing 4-game average for 'pass', 'pass_oe', and 'total_plays', shifted by one additional game to exclude the current one
    #print(df.head(2))
    df['trailing_pass_avg'] = df['pass_rate'].shift(1).rolling(window=trailing_weeks).mean()
    df['trailing_pass_oe_avg'] = df['pass_oe'].shift(1).rolling(window=trailing_weeks).mean()
    df['trailing_total_plays_avg'] = df['total_plays'].shift(1).rolling(window=trailing_weeks).mean()

# Shift the columns to get the values of the next game for 'pass', 'pos_team_total', and 'total_line'
    df['next_game_pass'] = df['pass_total'].shift(-1)
    df['next_game_pos_team_total'] = df['pos_team_total'].shift(-1)
    df['next_game_total_line'] = df['total_line'].shift(-1)

# Drop rows where we do not have a full 4-game average or the next game's values
    df_trailing_averages = df.dropna(subset=['trailing_pass_avg', 'trailing_pass_oe_avg', 'trailing_total_plays_avg'])

# Select only the relevant columns in the final dataframe
    all_data = df_trailing_averages[['game_id','trailing_pass_avg', 'trailing_pass_oe_avg', 'trailing_total_plays_avg', 
                                             'posteam','pass_total', 'pos_team_total', 'total_line']]
    all_data['identifier'] = all_data['game_id']+'_pos_'+all_data['posteam']
    return all_data

In [ ]:
defense_df

In [ ]:
defense_games = data_creator(defense_df,4)

In [ ]:

# Assuming df is your original dataframe with relevant columns and it's sorted by game date
# Calculate the trailing 4-game average for 'pass', 'pass_oe', and 'total_plays', shifted by one additional game to exclude the current one
df['trailing_pass_avg'] = df['pass_rate'].shift(1).rolling(window=4).mean()
df['trailing_pass_oe_avg'] = df['pass_oe'].shift(1).rolling(window=4).mean()
df['trailing_total_plays_avg'] = df['total_plays'].shift(1).rolling(window=4).mean()

# Shift the columns to get the values of the next game for 'pass', 'pos_team_total', and 'total_line'
df['next_game_pass'] = df['pass_total'].shift(-1)
df['next_game_pos_team_total'] = df['pos_team_total'].shift(-1)
df['next_game_total_line'] = df['total_line'].shift(-1)

# Drop rows where we do not have a full 4-game average or the next game's values
df_trailing_averages = df.dropna(subset=['trailing_pass_avg', 'trailing_pass_oe_avg', 'trailing_total_plays_avg'])

# Select only the relevant columns in the final dataframe
all_data = df_trailing_averages[['game_id','posteam','trailing_pass_avg', 'trailing_pass_oe_avg', 'trailing_total_plays_avg', 
                                             'pass_total', 'pos_team_total', 'total_line']]


In [ ]:
all_data['identifier'] = all_data['game_id']+'_pos_'+all_data['posteam']

In [ ]:
defense_data = defense_games.rename(columns={'trailing_pass_avg':'def_pass_rate','trailing_pass_oe_avg':'def_pass_oe','trailing_total_plays_avg':'def_total_plays'})[['identifier','def_pass_rate','def_pass_oe','def_total_plays']]

In [ ]:
merged_data = all_data.merge(defense_data,on='identifier')

In [ ]:

# Prepare the features (X) and target (y)

X = merged_data[['trailing_pass_avg', 'trailing_pass_oe_avg', 'trailing_total_plays_avg','total_line','pos_team_total','def_pass_rate','def_pass_oe','def_total_plays']]
y = merged_data['pass_total']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Make predictions with the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

# Feature importance
importance = best_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

# Print the feature importance
print("Feature Importance:")
print(feature_importance_df)


In [ ]:
import pickle


with open('pass_volume_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)